In [ ]:
#scrapbook
def create_enumeration_area_map(row):
    """
    Create a map for a single enumeration area showing:
    1. The nearest road point
    2. The grid cell geometry
    3. The centroid
    
    Parameters:
    row: A single row from the results DataFrame
    """
    import folium
    from shapely import wkt
    
    # Convert WKT string back to geometry
    grid_geometry = wkt.loads(row.geometry)
    
    # Create a map centered on the nearest road point (if it exists) or centroid
    if row.nearest_road_lat and row.nearest_road_lon:
        center_lat, center_lon = row.nearest_road_lat, row.nearest_road_lon
    else:
        center_lat, center_lon = row.centroid_lat, row.centroid_lon
        
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=17,
        tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
        attr='Google Maps'
    )
    
    # Add the grid cell geometry
    folium.GeoJson(
        grid_geometry.__geo_interface__,
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': '#ff0000',
            'weight': 2,
            'fillOpacity': 0.1
        },
        popup=f"Grid ID: {row.grid_id}"
    ).add_to(m)
    
    # Add the centroid point
    folium.CircleMarker(
        location=[row.centroid_lat, row.centroid_lon],
        radius=6,
        color='blue',
        fill=True,
        popup='Centroid'
    ).add_to(m)
    
    # Add the nearest road point if it exists
    if row.nearest_road_lat and row.nearest_road_lon:
        folium.CircleMarker(
            location=[row.nearest_road_lat, row.nearest_road_lon],
            radius=6,
            color='red',
            fill=True,
            popup=f'Nearest Road Point (Distance: {row.distance_to_road:.1f}m)'
        ).add_to(m)
        
        # Add a line connecting centroid to nearest road point
        folium.PolyLine(
            locations=[
                [row.centroid_lat, row.centroid_lon],
                [row.nearest_road_lat, row.nearest_road_lon]
            ],
            weight=2,
            color='red',
            opacity=0.8
        ).add_to(m)
    
    # Add a legend
    legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; 
         background-color: white; padding: 10px; border: 2px solid grey; border-radius: 5px;">
        <p><i style="background: red; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Nearest Road</p>
        <p><i style="background: blue; width: 10px; height: 10px; display: inline-block; border-radius: 50%;"></i> Centroid</p>
        <p><i style="border: 2px solid red; width: 10px; height: 10px; display: inline-block;"></i> Grid Boundary</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    return m

# Create maps for all enumeration areas
for idx, row in results_df.iterrows():
    m = create_enumeration_area_map(row)
    m.save(f'enumeration_area_map_{row.grid_id}.html')

# Or test with just one area
test_map = create_enumeration_area_map(results_df.iloc[0])
test_map.save('test_enumeration_area_map.html')